# Task 1 - Input Data

This notebook will aim to analyse the raw data that we want to import into our system. We will perform exploratory data analysis on the data, identify irregularities and potential problems and ultimately clean the data so that we can avoid bad data entering our database.

### Import Data

First we can import our necessary libraries:

In [654]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.impute import KNNImputer


Next we can load our data and view the first few rows:

In [655]:
# Load data and view top rows
df = pd.read_csv('import_data.csv')
df.head()

,mortgage_advisor_name,consumer_1_email,consumer_2_email,consumer_1_email_validity,consumer_2_email_validity,consumer_1_name,consumer_2_name,consumer_1_phone_number,consumer_2_phone_number,consumer_1_phone_number_validity,...,mortgage_type,mortgage_payment_type,mortgage_lender,mortgage_loan_purpose,mortgage_scheme,mortgage_advisor_email,mortgage_account_number,mortgage_description,mortgage_charge,mortgage_pct_owned
0,Vanessa Abigail,way_country_able@example.com,NaN,Valid,NaN,Rachel Daniel,NaN,7.822269e+09,NaN,Valid,...,owner-occupied,repayment,Leeds BS,purchase,fixed,art_miss_hit@example.com,NaN,NaN,first-charge,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Vanessa Abigail,have_chap_slight@example.com,sun_play_green@example.com,Valid,Valid,Laura Kathryn,Sam Wayne,7.848853e+09,7.576772e+09,Valid,...,owner-occupied,repayment,Santander UK Plc,further-advance,fixed,art_miss_hit@example.com,NaN,NaN,first-charge,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Vanessa Abigail,leg_some_cause@example.com,town_plus_bloke@example.com,Valid,Risky,Benjamin Anthony,Adrian Jenni,7.814313e+09,7.531585e+09,Valid,...,owner-occupied,repayment,Santander UK Plc,remortgage,fixed,art_miss_hit@example.com,NaN,NaN,first-charge,NaN


We can see that this data looks very messy with a lot of missing values. 

We can look further into this by looking at a description of our data:

In [656]:
display(df.describe().T)
print(df.shape)

,count,mean,std,min,25%,50%,75%,max
consumer_1_phone_number,3310.0,7.780850e+09,1.506626e+08,7.511162e+09,7.599251e+09,7.813421e+09,7.898214e+09,7.999995e+09
consumer_2_phone_number,2000.0,7.787510e+09,1.494500e+08,7.511132e+09,7.714090e+09,7.819519e+09,7.917412e+09,7.999895e+09
consumer_1_correspondence_postcode,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_2_correspondence_postcode,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
property_value,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mortgage_preferential_rate,3300.0,2.360197e+00,1.448875e+01,8.500000e-01,1.740000e+00,1.990000e+00,2.300000e+00,8.336700e+02
mortgage_loan_amount,3322.0,1.591385e+05,6.196074e+04,2.440000e+02,1.181488e+05,1.517670e+05,1.941412e+05,5.250000e+05
mortgage_property_value,2229.0,2.865802e+05,1.015140e+05,7.500000e+04,2.150000e+05,2.700000e+05,3.380070e+05,1.050000e+06
mortgage_account_number,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mortgage_description,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(11080, 31)


We can see that some our our columns are completely empty. Therefore we can create a function to remove these columns:

In [657]:
def drop_empty_columns(df):
    # Get the columns before dropping empty columns
    cols_before = df.columns

    # Use the dropna() function to drop columns with all NaN (empty) values
    df.dropna(axis=1, how='all', inplace=True)

    # Get the columns after dropping empty columns
    cols_after = df.columns

    # Find the removed columns
    removed_columns = set(cols_before) - set(cols_after)
    return df, removed_columns

df, removed_cols = drop_empty_columns(df)
print('Removed columns:', removed_cols)
print(df.shape)

Removed columns: {'mortgage_account_number', 'mortgage_description', 'consumer_2_correspondence_postcode', 'property_value', 'consumer_1_correspondence_postcode'}
(11080, 26)


We can see that we have removed six columns that are useless to us since they contain no data. However, in a proper business setting we would need to evaluate the importance of these columns and investigate why we aren't collecting any information for them.

We can similarly create a function to remove any completely blank rows:

In [658]:
def drop_empty_rows(df):
    # Get the number of rows before dropping empty rows
    rows_before = df.shape[0]

    # Use the dropna() function to drop rows with all NaN (empty) values
    df.dropna(axis=0, how='all', inplace=True)

    # Get the number of rows after dropping empty rows
    rows_after = df.shape[0]

    # Calculate the number of rows removed
    rows_removed = rows_before - rows_after
    return df, rows_removed

df, removed_rows = drop_empty_rows(df)
print('Number of removed rows:', removed_rows)

Number of removed rows: 7756


We can again look into a description of our data:

In [659]:
display(df.describe(include='all').T)
print(df.shape)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
mortgage_advisor_name,3324,21,Eleanor Sarah,446,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_1_email,3185,2919,opportunity_study_find@example.com,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_2_email,1756,1596,function_gas_britain@example.com,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_1_email_validity,3185,3,Valid,3059,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_2_email_validity,1756,3,Valid,1669,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_1_name,3324,2937,Amy Gurjit,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_2_name,2423,2120,Craig Elinor,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_1_phone_number,3310.0,NaN,NaN,NaN,7780850386.466767,150662550.346105,7511161541.0,7599250802.0,7813421438.0,7898213662.0,7999994963.0
consumer_2_phone_number,2000.0,NaN,NaN,NaN,7787510048.9545,149449964.979327,7511132246.0,7714089587.25,7819518657.0,7917411756.0,7999895376.0
consumer_1_phone_number_validity,3310,2,Valid,3299,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(3324, 26)


We can see that the column mortgage_charge only has one unique value which makes the row completely redundant. Therefore, we can also remove this row:

In [660]:
# Drop column
df = df.drop('mortgage_charge', axis=1)
print(df.shape)

(3324, 25)


We can also check to see if our dataset has any duplicated values and remove them if so:

In [661]:
print('Number of duplicate rows:', df.duplicated().sum())

# Drop duplicates
df = df.drop_duplicates()
print(df.shape)

Number of duplicate rows: 217
(3107, 25)


We can look at our description of our data again to notice any other irregularities: 

In [662]:
display(df.describe(include='all').T)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
mortgage_advisor_name,3107,21,Eleanor Sarah,446,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_1_email,3007,2919,reason_time_best@example.com,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_2_email,1643,1596,it_summer_horse@example.com,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_1_email_validity,3007,3,Valid,2895,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_2_email_validity,1643,3,Valid,1560,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_1_name,3107,2937,Rachel Joshua,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_2_name,2243,2120,Robert Daniel,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_1_phone_number,3094.0,NaN,NaN,NaN,7779464763.443439,150495601.096449,7511161541.0,7598388748.75,7812347416.0,7896909857.0,7999994963.0
consumer_2_phone_number,1841.0,NaN,NaN,NaN,7787649939.15698,148676740.718385,7511132246.0,7714789653.0,7819784286.0,7915588691.0,7999895376.0
consumer_1_phone_number_validity,3094,2,Valid,3084,NaN,NaN,NaN,NaN,NaN,NaN,NaN


One thing of note is how there is one more mortgage_advisor_email than mortgage_advisor_name where we woud assume that these should be equal. We can look deeper into this:

In [663]:
def show_multiple_records_by_name(df, name_col, record_col):

    # Group by names and count the number of unique emails for each name 
    name_record_counts = df.groupby(name_col)[record_col].nunique()

    # Filter names with two or more unique emails
    names_with_multiple_records = name_record_counts[name_record_counts >= 2].index.tolist()

    return names_with_multiple_records 

print('Names with two or more emails assigned:', show_multiple_records_by_name(df, 'mortgage_advisor_name', 'mortgage_advisor_email'))

Names with two or more emails assigned: ['Olivia Joshua']


We can see that Olivia Joshua is the person with two emails associated with her name, we can look further into this:

In [664]:
df[df['mortgage_advisor_name']=='Olivia Joshua'][['mortgage_advisor_name', 'mortgage_advisor_email']].value_counts()

mortgage_advisor_name  mortgage_advisor_email          
Olivia Joshua          great_judge_relation@example.com    249
                       air_since_particular@example.com     36
Name: count, dtype: int64

We can see that one email for this mortgage advisor is a lot more prominent in the dataset. Although it is possible that she uses two emails, it is best that we update the less prominent email to her more prominent. However, first we should check that no one else uses this email and there has been a mix up on the name:

In [665]:
df[df['mortgage_advisor_email']=='air_since_particular@example.com']['mortgage_advisor_name'].value_counts()

mortgage_advisor_name
Olivia Joshua    36
Name: count, dtype: int64

Since no one else uses this email, we can confidently say that Olivia Joshua is the correct mortgage advisor here and we update her email:

In [666]:
def replace_with_most_popular_email(df, name, name_col, email_col):

    # Get most common email
    most_common_email = df[df[name_col]==name][[email_col]].value_counts().index[0][0]

    # We can update the email to keep it consistent
    df.loc[df[name_col]== name, email_col] = most_common_email

    return df

df = replace_with_most_popular_email(df, 'Olivia Joshua', 'mortgage_advisor_name', 'mortgage_advisor_email')

We can check that this has updated Olivia Josuhua's emails:

In [667]:
df[df['mortgage_advisor_name']=='Olivia Joshua'][['mortgage_advisor_name', 'mortgage_advisor_email']].value_counts()

mortgage_advisor_name  mortgage_advisor_email          
Olivia Joshua          great_judge_relation@example.com    285
Name: count, dtype: int64

Now that we are happy with our mortgage advisors, we can look into other columns. First we will look into DOB as we can already see above that the date formats are not consistent:

In [668]:
def convert_date_format(input_date):
    if pd.isna(input_date):
        return input_date  # Return NaN as is
    else:
        # Define the input and output date formats
        input_format = "%A, %d %B %Y"
        output_format = "%d/%m/%Y"

        # Parse the input date using the input format
        date_obj = datetime.strptime(input_date, input_format)

        # Convert the date object to the desired output format
        output_date = date_obj.strftime(output_format)
        return output_date

df['consumer_2_dob'] = df['consumer_2_dob'].apply(convert_date_format)


We can sense check these date of births:

In [687]:
print('Min. consumer 1 DOB:', pd.to_datetime(df['consumer_1_dob']).min())
print('Min. consumer 2 DOB:', pd.to_datetime(df['consumer_2_dob']).min())

Min. consumer 1 DOB: 1920-01-07 00:00:00


C:\Users\samke\AppData\Local\Temp\ipykernel_21084\3153728012.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  print('Min. consumer 2 DOB:', pd.to_datetime(df['consumer_2_dob']).min())


DateParseError: Unknown datetime string format, unable to parse: No second consumer, at position 0

We can see that our oldest consumers are both over 100 years old. Since this is possible we will keep the values but with more time we may want to look deeper into these records to ensure they are still relevant. 

We can now look into how many consumers have multiple date of births to their name:

In [670]:
print('Number of consumers that have multiple date of births to their name (consumer 1):', len(show_multiple_records_by_name(df, 'consumer_1_name', 'consumer_1_dob')))
print('Number of consumers that have multiple date of births to their name (consumer 2):', len(show_multiple_records_by_name(df, 'consumer_2_name', 'consumer_2_dob')))

Number of consumers that have multiple date of births to their name (consumer 1): 82
Number of consumers that have multiple date of births to their name (consumer 2): 44


We can see that some consumers have multiple DOBs which is illogical. This is likely due to consumers sharing names so we need to create a consumer ID for each consumer so that we can distinguish between these consumers: 

In [671]:
df['consumer_1_ID'] = df['consumer_1_name'] + '_' + df['consumer_1_dob'].astype(str) 
df['consumer_2_ID'] = df['consumer_2_name'] + '_' + df['consumer_2_dob'].astype(str)

In [672]:
print('Number of consumer IDs that have multiple date of births to their ID:', len(show_multiple_records_by_name(df, 'consumer_1_ID', 'consumer_1_dob')))
print('Number of consumer IDs that have multiple date of births to their ID:', len(show_multiple_records_by_name(df, 'consumer_2_ID', 'consumer_2_dob')))

Number of consumer IDs that have multiple date of births to their ID: 0
Number of consumer IDs that have multiple date of births to their ID: 0


Now after re-running the code with ID instead of name, we can see above that there are no customers with multiple DOBs. We can also check that no customer have multiple emails or phone numbers for each ID:

In [673]:
print('Consumers with multiple emails (consumer 1): ', show_multiple_records_by_name(df, 'consumer_1_ID', 'consumer_1_email'))
print('Consumers with multiple emails (consumer 2): ', show_multiple_records_by_name(df, 'consumer_2_ID', 'consumer_2_email'))
print('Consumers with multiple phone numbers (consumer 1): ', show_multiple_records_by_name(df, 'consumer_1_ID', 'consumer_1_phone_number'))
print('Consumers with multiple phone numbers (consumer 2): ', show_multiple_records_by_name(df, 'consumer_2_ID', 'consumer_2_phone_number'))

Consumers with multiple emails (consumer 1):  []
Consumers with multiple emails (consumer 2):  ['Rochelle Imran_28/01/1968']
Consumers with multiple phone numbers (consumer 1):  []
Consumers with multiple phone numbers (consumer 2):  ['Rochelle Imran_28/01/1968']


We can see there is only one consumer where this is the case so we can look into this consumer:

In [674]:
df[df['consumer_2_ID'] == 'Rochelle Imran_28/01/1968'][['consumer_1_name', 'consumer_2_name', 'consumer_1_email', 'consumer_2_email', 'consumer_1_phone_number', 'consumer_2_phone_number']]

,consumer_1_name,consumer_2_name,consumer_1_email,consumer_2_email,consumer_1_phone_number,consumer_2_phone_number
551,Samantha Jack,Rochelle Imran,small_miss_past@example.com,clear_back_best@example.com,7.859950e+09,7.582672e+09
3228,Samantha Jack,Rochelle Imran,small_miss_past@example.com,small_miss_past@example.com,7.859950e+09,7.859950e+09


We can see that this is simply an error where the consumer 1's email and phone has been inputted into the consumer 2 field. Therefore, we can ammend this:

In [675]:
df.loc[df['consumer_2_ID'] == 'Rochelle Imran_28/01/1968', 'consumer_2_email'] = 'clear_back_best@example.com'
df.loc[df['consumer_2_ID'] == 'Rochelle Imran_28/01/1968', 'consumer_2_phone_number'] = '7582672214'

In [676]:
print('Consumers with multiple emails (consumer 1): ', show_multiple_records_by_name(df, 'consumer_1_ID', 'consumer_1_email'))
print('Consumers with multiple emails (consumer 2): ', show_multiple_records_by_name(df, 'consumer_2_ID', 'consumer_2_email'))
print('Consumers with multiple phone numbers (consumer 1): ', show_multiple_records_by_name(df, 'consumer_1_ID', 'consumer_1_phone_number'))
print('Consumers with multiple phone numbers (consumer 2): ', show_multiple_records_by_name(df, 'consumer_2_ID', 'consumer_2_phone_number'))

Consumers with multiple emails (consumer 1):  []
Consumers with multiple emails (consumer 2):  []
Consumers with multiple phone numbers (consumer 1):  []
Consumers with multiple phone numbers (consumer 2):  []


Re-running the above code we can see that there are no longer any consumers with multiple emails or phone numbers to their name.

Since we have missing values for consumer emails and consumer phone numbers, we can see if we can impute these from the consumer ID:

In [677]:
def replace_missing_email_phone(df):
    # Get null counts before replacing missing values
    email_null_count_1 = df["consumer_1_email"].isnull().sum()
    email_null_count_2 = df["consumer_2_email"].isnull().sum()
    phone_null_count_1 = df["consumer_1_phone_number"].isnull().sum()
    phone_null_count_2 = df["consumer_2_phone_number"].isnull().sum()

    # Forward-fill missing values for email and phone number columns within each group
    df["consumer_1_email"] = df.groupby("consumer_1_ID")["consumer_1_email"].transform(lambda x: x.fillna(method="ffill"))
    df["consumer_2_email"] = df.groupby("consumer_2_ID")["consumer_2_email"].transform(lambda x: x.fillna(method="ffill"))
    df["consumer_1_phone_number"] = df.groupby("consumer_1_ID")["consumer_1_phone_number"].transform(lambda x: x.fillna(method="ffill"))
    df["consumer_2_phone_number"] = df.groupby("consumer_2_ID")["consumer_2_phone_number"].transform(lambda x: x.fillna(method="ffill"))

    # Get null counts after replacing missing values
    print(email_null_count_1 - df["consumer_1_email"].isnull().sum(), 'missing emails replaced (consumer 1)')
    print(email_null_count_2 - df["consumer_2_email"].isnull().sum(), 'missing emails replaced (consumer 2)')
    print(phone_null_count_1 - df["consumer_1_phone_number"].isnull().sum(), 'missing phone numbers replaced (consumer 1)')
    print(phone_null_count_2 - df["consumer_2_phone_number"].isnull().sum(), 'missing phone numbers replaced (consumer 2)')

    return df

df = replace_missing_email_phone(df)

0 missing emails replaced (consumer 1)
0 missing emails replaced (consumer 2)
0 missing phone numbers replaced (consumer 1)
0 missing phone numbers replaced (consumer 2)


We can see that we were unfortunately unable to impute any missing emails and phone_numbers from their associated pairs. We could spemd longer looking into this data to try and and impute a consumer's name or email from other data, however due to time constraints we will simply replace any missing value with 'Unknown'. We can also update the email_validity and phone_validity columns to add a third value 'No email' or 'No phone number' respectively:

In [678]:
# Fill nan emails and phone numbers with Unknown
df['consumer_1_email'].fillna('Unknown', inplace=True)
df['consumer_2_email'].fillna('Unknown', inplace=True)
df['consumer_1_phone_number'].fillna('Unknown', inplace=True)
df['consumer_2_phone_number'].fillna('Unknown', inplace=True)

# Update validity columns
df.loc[df['consumer_1_email']=='Unknown', 'consumer_1_email_validity'] = 'No email'
df.loc[df['consumer_2_email']=='Unknown', 'consumer_2_email_validity'] = 'No email'
df.loc[df['consumer_1_phone_number']=='Unknown', 'consumer_1_phone_number_validity'] = 'No phone number'

We can also assume that any missing values for consumer_2_name are due to there being no second consumer. Therefore, we can replace all NaN values with 'No second consumer' and update the ID, DOB, email and phone number to say the same:

In [679]:
# Fill NaN columns
df['consumer_2_name'].fillna('No second consumer', inplace=True)

# Update other columns
df.loc[df['consumer_2_name']=='No second consumer', 'consumer_2_dob'] = 'No second consumer'
df.loc[df['consumer_2_name']=='No second consumer', 'consumer_2_email'] = 'No second consumer'
df.loc[df['consumer_2_name']=='No second consumer', 'consumer_2_phone_number'] = 'No second consumer'
df.loc[df['consumer_2_name']=='No second consumer', 'consumer_2_ID'] = 'No second consumer'

We can look at a description of our data again:

In [680]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
mortgage_advisor_name,3107,21,Eleanor Sarah,446,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_1_email,3107,2920,Unknown,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_2_email,3107,1597,No second consumer,864,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_1_email_validity,3107,4,Valid,2895,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_2_email_validity,3107,4,Valid,1560,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_1_name,3107,2937,Rachel Joshua,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_2_name,3107,2121,No second consumer,864,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_1_phone_number,3107,3009,Unknown,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_2_phone_number,3107,1788,No second consumer,864,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_1_phone_number_validity,3107,3,Valid,3084,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We are still missing values for several columns. Some of these could be of importance and should be filled with further research, however due to time constraints we will fill with 'Unknown' for several of these columns:

In [681]:
# Fill NaN columns
df[['consumer_2_dob', 'property_address', 'mortgage_start_date']] = df[['consumer_2_dob', 'property_address', 'mortgage_start_date']].fillna('Unknown')


We can look into the value counts of mortgage_pct_owned:

In [682]:
df['mortgage_pct_owned'].value_counts()

mortgage_pct_owned
0.0    422
Name: count, dtype: int64

Since this column only contains one value we can consider it redundant and drop it:

In [683]:
df.drop('mortgage_pct_owned', axis=1, inplace=True)

For our remaining columns with missing values, these are all fairly important features such as the mortgage property value and the mortgage preferential rate. If we wanted our dataset to be as accurate as possible we should do some further research into these features to find the missing values or mark them as Unknown so that we know that this data was missing if we were to look into particular cases at a later date. 

However, if we wanted to perform some general analysis on our dataset as a whole and the complete accuracy of our data on a case by case basis was of less importance, we could impute these missing values based on the rest of the data in our dataset. 

We will assume the latter and impute the missing values. To do this we will use replace the categorical features with the most common value and use a KNN imputer to fill the missing numerical values: 

In [684]:
# Get a list of categorical features
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

# Fill missing values in categorical columns with the mode of each column
for feature in categorical_features:
    df[feature] = df[feature].fillna(df[feature].mode().iloc[0])

In [685]:
# Get a list of numerical features
numerical_features = df.select_dtypes(include=['number']).columns.tolist()

# Impute missing values
imputer = KNNImputer(n_neighbors=5)
df[numerical_features] = imputer.fit_transform(df[numerical_features])


We can check that our dataset contains no more missing values:

In [686]:
print('Total missing values:', df.isna().sum().sum())

Total missing values: 0


Finally we can save to a csv file:

In [ ]:
df.to_csv('import_data_clean.csv', index=False)

We can also run the whole cleaning process through a Python module: 

In [688]:
from task1_pipeline import clean_data

clean_data('import_data.csv', 'import_data_clean.csv')

0 missing emails replaced (consumer 1)
0 missing emails replaced (consumer 2)
0 missing phone numbers replaced (consumer 1)
0 missing phone numbers replaced (consumer 2)
Total missing values: 0
